# 코루틴(coroutine)
제너레이터를 코루틴으로 활용할 수 있음  
비동기 프로그래밍의 기초를 지원하기 위해 제너레이터가 어떻게 코루틴으로 진화하는 지 살펴보기  

코루틴을 지원하기 위해 추가된 기본 메서드
- .close()
- .throw(ex_type[, ex_value[, ex_traceback]])
- .send(value)

## 제너레이터 인터페이스의 메서드  
위의 코루틴을 위한 메서드가 각각 무엇인지, 어떻게 작동하는지, 어떻게 사용되는 지 확인  
간단한 코루틴 생성  
코루틴의 고급 사용법과 서브 제너레이터(코루틴)에 위임을 통해 리팩토링하는 방법  
여러 코루틴을 조합하는 방법 학습  

### close()
이 메서드를 호출하면 제너레이터에서 GeneratorExit 예외 발생  
GeneratorExit 예외 미처리 시 제너레이터가 더 이상 값을 생성하지 않으며 반복이 중지됨  
- GeneratorExit 예외는 종료 상태를 지정하는 데 사용 가능
- 코루틴이 일종의 자원관리를 하는 경우 이 예외를 통해 코루틴이 보유한 모든 자원 해제 가능 
- 컨텍스트 관리자를 사용하거나 finally 블록에 코드를 배치하는 것과 유사하나 GeneratorExit 예외를 사용하면 보다 명확하게 처리 가능

아래 코드는 코루틴을 사용하여 데이터베이스 연결을 유지한 상태에서 한 번에 모든 레코드를 읽는 대신에 특정 크기의 페이지를 스트리밍함
```python
def stream_db_records(db_handler):
    try:
        while True:
            yield db_handler.read_n_records(10)
            
    except GeneratorExit:
        db_handler.close()
```

```python
>> streamer = stream_db_records(DBHandler("testdb"))
>> next(streamer)
```
>[(0, 'row 0'), (1, 'row 1'), (2, 'row 2'), (3, 'row 3'), ...]
```python
>> next(streamer)
```
>[(0, 'row 0'), (1, 'row 1'), (2, 'row 2'), (3, 'row 3'), ...]
```python
>> streamer.close()
```
> INFO:...:closing connection to database 'testdb'

이처럼 제너레이터를 호출할  때마다 데이터베이스 핸들러에서 얻은 10개의 레코드를 반환하고, 명시적으로 반복을 끝내고 close()를 호출하면 데이터베이스 연결되 함께 종료됨

### throw(ex_type[, ex_value[, ex_traceback]])
이 메서드는 현재 제너레이터가 중단된 위치에서 예외를 던짐  
제너레이터가 예외를 처리했으면 해당 except 절에 있는 코드가 호출되고, 예외를 처리하지 않았으면 예외가 호출자에게 전파됨  

```python
class CustomerException(Exception):
    pass
    
def stream_data(db_handler):
    while True:
        try:
            yield db_handler.read_n_records(10)
        except CustomException as e:
            logger.warning("처리 가능한 에러 %r, 계속 진행", e)
        except Exception as e:
            logger.error("처리할 수 없는 에러 %r, 중단", e)
            db_handler.close()
            break
```
CustomException을 처리하고 있으며 이 예외가 발생한 경우 제너레이터는 INFO 레벨의 메세지를 기록하고 다음 yield 구문으로 이동하여 데이터베이스에서 다시 데이터를 가져옴  
이 예제에서는 모든 예외를 처리하고 있으나 마지막 블록 (except Exception)이 없으면 제너레이터가 중지된 행에서 예외가 호출자에게 전파되고 제너레이터는 중지됨  


```python
>> streamer = stream_data(DBHandler("testdb"))
>> next(streamer)
```
>[(0, 'row 0'), (1, 'row 1'), (2, 'row 2'), (3, 'row 3'), ...]
```python
>> next(streamer)
```
>[(0, 'row 0'), (1, 'row 1'), (2, 'row 2'), (3, 'row 3'), ...]
```python
>> streamer.throw(CustomException)
```
> WARNING: 처리 가능한 에러 CustomException(), 계속 진행  
>[(0, 'row 0'), (1, 'row 1'), (2, 'row 2'), (3, 'row 3'), ...]
```python
>> streamer.throw(RuntimeError)
```
> ERROR: 처리할 수 없는 에러 RuntimeError(), 중단  
> INFO: 'testdb' 데이터베이스 연결 종료  
> Traceback (most recent call last):  
> ...  
> StopIteration

도메인에서 처리하고 있는 CustomException 예외를 받은 경우 제너레이터는 계속 진행됨  
그러나 그 외 예외는 Exception으로 넘어가서 데이터베이스 연결을 종료하고 반복도 종료하게 됨  
마지막에 StopIteration이 출력된 것에서 알 수 있듯이 이 제너레이터는 이제 더 이상 반복을 할 수 없음

### send(value)
위의 예제에서는 데이터베이스 레코드를 조회하는 제너레이터를 만들고 반복을 끝낼 때 데이터베이스 리소스를 해제함  
-> 제너레이터가 제공하는 close 메서드를 사용하는 좋은 예제이나 코루틴으로 보다 많은 일을 할 수 있음  

예제에서 제너레이트의 주요 기능: 고정된 수의 레코드 읽기  
읽어올 개수를 파라미터로 받아서 호출하도록 수정하려고 해도 next() 함수는 이러한 옵션을 제공하지 않음  
이럴 때 send() 메서드 사용 가능

In [3]:
def stream_db_records(db_handler):
    retrieved_data = None
    previous_page_size = 10
    try:
        while True:
            page_size = yield retrieved_data
            if page_size is None:
                page_size = previous_page_size
            
            previous_page_size = page_size
            
            retrieved_data = db_handler.read_n_records(page_size)
    except GeneratorExit:
        db_handler.close()

이제 send() 메서드를 통해 인자값 전달 가능  
send() 메서드는 **제너레이터와 코루틴을 구분하는 기준이 됨**  
send() 메서드를 사용했다는 것 = yield 키워드가 할당 구문의 오른쪽에 나오게 되고 인자값을 받아서 다른 곳에 할당할 수 있음  

코루틴에서는 일반적으로 다음과 같은 형태로 yield 키워드를 사용함  
```python
receive = yield produced
```

이 경우 yield 키워드는
1. 생성된 값을 호출자에게 보내고 그 곳에 멈추는 것 (호출자는 next() 메서드를 호출해 다음 라운드에서 값을 가져올 수 있음)
2. 거꾸로 호출자로부터 send() 메서드를 통해 전달된 생성된 값을 받는 것 (이렇게 입력된 값은 receive 변수에 할당됨)

코루틴에 값을 전송하는 것은 yield 구문이 멈춘 상태에서만 가능함  
이를 위해서는 코루틴을 해당 상태까지 이동시켜야하는데 해당 상태로 코루틴을 이동시키는 유일한 방법은 next()를 호출하는 것임  
-> **코루틴에게 무엇인가를 보내기 전에 next() 메서드를 적어도 한 번은 호출해야 함을 의미**  

next() 메서드를 호출하지 않으면 아래와 같은 예외 발생 

```python
>>> c = coro()
>>> c.send(1)
```
>Trackback (most recent call last):  
>...  
>TypeError: can't send non-None value to a just-started generator

앞에서 사용한 예제를 데이터베이스에서 읽을 레코드의 길이를 파라미터로 받도록 수정  
1. 제너레이터에서 처음 next()를 호출하면 yield를 포함하는 위치까지 이동하고 현재 상태의 변수 값을 반환하고 거기에 멈춤  
2. 변수의 초기 값이 None이므로 처음 next()를 호출하면 None을 반환  
3-1. 그냥 next()를 호출하면 기본값인 10을 사용하여 평소처럼 이후 작업이 계속됨. next()는 send(None)과 동일  
3-2. send(value)를 통해 명시적인 값을 제공하면 yield 문의 반환값으로 page_size 변수에 설정됨. 사용자가 지정한 값이 page_size로 설정됨

이후 호출에서도 동일한 로직이 적용되므로 아무 떄나 페이지 크기를 지정할 수 있게 됨  

위와 같은 사항을 반영하여 깔끔하게 코드를 고치면 아래와 같음

In [5]:
def stream_db_records(db_handler):
    retrieved_data = None
    page_size = 10
    try:
        while True:
            page_size = (yield retrieved_data) or page_size
            retrieved_data = db_handler.read_n_records(page_size)
    except GeneratorExit:
        db_handler.close()

이 코드도 기대한 대로 잘 동작하나 역시 send() 전에 next()를 먼저 호출해야 함 -> 호출하지 않으면 TypeError 발생  

next()를 반드시 호출해야 한다는 것을 기억할 필요 없이 코루틴을 생성하자마자 바로 사용할 수 있다면 훨씬 편할 것임  
아래 @prepare_coroutine 데코레이터는 코루틴을 좀 더 편리하게 사용하도록 자동으로 초기화를 해줌

In [7]:
def prepare_coroutine(coroutine):
    def wrapped(*args, **kwargs):
        advanced_coroutine = coroutine(*args, **kwargs)
        next(advanced_coroutine)
        return advanced_coroutine

    return wrapped

@prepare_coroutine
def stream_db_records(db_handler):
    retrieved_data = None
    page_size = 10
    try:
        while True:
            page_size = (yield retrieved_data) or page_size
            retrieved_data = db_handler.read_n_records(page_size)
    except GeneratorExit:
        db_handler.close()

```python
>> streamer = stream_db_records(DBHandler("testdb"))
>> len(streamer.send(5))
```
>5